![MSE Logo](https://moodle.msengineering.ch/pluginfile.php/1/core_admin/logocompact/300x300/1613732714/logo-mse.png "MSE Logo") 

# Latent Semantic Analysis with Gensim

In [38]:
import nltk
import pandas as pd
from TextPreprocessor import TextPreprocessor
from gensim import models, corpora, similarities
import json

from nltk.corpus import stopwords, wordnet

In [62]:
content = {}

# with open("../../rss/content_daily.json") as f:
#     content.update(json.load(f))
# with open("../../rss/content_huffpost.json") as f:
#     content.update(json.load(f))
with open("../../rss/content_guardain.json") as f:
    content.update(json.load(f))
# with open("../../archive_article_list_05_12/content_daily.json") as f:
#     content.update(json.load(f))
with open("../../archive_article_list_05_12/content_guardain.json") as f:
    content.update(json.load(f))
# with open("../../archive_article_list_05_12/content_huffpost.json") as f:
#     content.update(json.load(f))
text = list(content.values())

In [63]:
data_df = pd.DataFrame({'text': text})

## Data preprocessing

You will need first to preprocess the data through the following stages:
1. tokenization
2. stopword removal
2. POS-based filtering (optional)
3. lemmatization or stemming (optional)
4. addition of bigrams to each document (optional)
5. filtering of infrequent words
6. inspection and filtering of frequent words

In [64]:
language = 'english'
stop_words = set(stopwords.words(language))
stop_words = list(stop_words).extend(['\"', '\'', '\'\'', '`', '``', '\'s',"'",','])

processor = TextPreprocessor(
    language = language,
    pos_tags = {"n","j"},
    stopwords = stop_words,
    lemmatize=True,
    remove_numbers=True,
    punctuations=['.', ',', ':', ';', '?', '!', '"', '\'', '`', '``', '\'s','"'],
)

In [65]:
data_df['processed'] = processor.transform(data_df['text'])

In [66]:
data_df['tokenized'] = data_df['processed'].apply(nltk.word_tokenize)

In [7]:
data_df.head()

,text,processed,tokenized
0,Crowds covered in Union Jack flags have gather...,crowd union jack flag mall today secure prime ...,"[crowd, union, jack, flag, mall, today, secure..."
1,As royal watchers around the world celebrate t...,royal watcher around world celebrate queen pla...,"[royal, watcher, around, world, celebrate, que..."
2,A Chinese trader has won a discrimination case...,trader discrimination case bos could admit thi...,"[trader, discrimination, case, bos, could, adm..."
3,The Atlanta activist Black Lives Matter has ap...,atlanta activist life matter straighten financ...,"[atlanta, activist, life, matter, straighten, ..."
4,Surreal images of a lion that lived in a Londo...,surreal image lion london show steak drinking ...,"[surreal, image, lion, london, show, steak, dr..."


In [67]:
words_to_filter = {"u","two","could","mr","mp","one", "per", "cent","hrt","--","the","i","'s","...","..","u","iii","'","it","if","would","could","may","should","will","can","that","like","use","need", "no","put","we","you","he","she","us"}

data_df["filter"] = data_df["tokenized"].apply(lambda x: [word.replace("'","").replace(",","").replace(".","") for word in x]).apply(lambda x: [word for word in x if not word.isnumeric()]).apply(lambda x: [word for word in x if word not in words_to_filter]).apply(lambda x: [word for word in x if not word[:2] == "ps"])

In [9]:
vocabulary = data_df['filter'].sum()

most_common = [word for word, freq in nltk.FreqDist(vocabulary).items() if freq > 4 and word not in words_to_filter]

print(f"Vocabulary length: {len(nltk.FreqDist(vocabulary))}")
print(f"Most common words: {len(most_common)}")

data_df.head()

Vocabulary length: 59652
Most common words: 22418


,text,processed,tokenized,filter
0,Crowds covered in Union Jack flags have gather...,crowd union jack flag mall today secure prime ...,"[crowd, union, jack, flag, mall, today, secure...","[crowd, union, jack, flag, mall, today, secure..."
1,As royal watchers around the world celebrate t...,royal watcher around world celebrate queen pla...,"[royal, watcher, around, world, celebrate, que...","[royal, watcher, around, world, celebrate, que..."
2,A Chinese trader has won a discrimination case...,trader discrimination case bos could admit thi...,"[trader, discrimination, case, bos, could, adm...","[trader, discrimination, case, bos, admit, thi..."
3,The Atlanta activist Black Lives Matter has ap...,atlanta activist life matter straighten financ...,"[atlanta, activist, life, matter, straighten, ...","[atlanta, activist, life, matter, straighten, ..."
4,Surreal images of a lion that lived in a Londo...,surreal image lion london show steak drinking ...,"[surreal, image, lion, london, show, steak, dr...","[surreal, image, lion, london, show, steak, dr..."


In [68]:
import pickle

with open("guardian_dataframe.p", "wb") as f:
    pickle.dump(data_df,f)

In [69]:
import pyLDAvis
import pyLDAvis.gensim_models 


def get_corpus(filtered_texts):
    dictionary = corpora.Dictionary(filtered_texts)
    corpus = [dictionary.doc2bow(text) for text in filtered_texts]

    return corpus


with open("all_sites_dataframe.p", "rb") as f:
    all_df = pickle.load(f)

with open("lsa_model.p","rb") as f:
    lsa_model = pickle.load(f)

corpus = get_corpus(data_df["filter"])
id2word = corpora.Dictionary(all_df['filter'])

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lsa_model, corpus, id2word)

with open("guardian_topics.json","w") as f:
    f.write(vis.to_json())

/Users/jeandemeusy/Desktop/antede/venv/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/jeandemeusy/Desktop/antede/venv/lib/python3.10/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/jeandemeusy/Desktop/antede/venv/lib/python3.10/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/jeandemeusy/Desktop/antede/venv/lib/python3.10/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/jeandemeusy/Deskto

In [76]:
with open("guardian_dataframe.p", "rb") as f:
    df = pickle.load(f)

len(df)

2679